In [34]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from video_module import Video,EmotionDetector

In [35]:
#LOSS AND ACCURACY PLOT

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('plot.png')
    plt.show()

In [36]:
#Data gfenerators


train_dir = 'C:/Users/athen/OneDrive/Documents/Mini project 2/archive (1)/images/images/train'
val_dir = 'C:/Users/athen/OneDrive/Documents/Mini project 2/archive (1)/images/images/test'

num_train = 28709
num_val = 7178
batch_size = 64
num_epoch = 10

datagen = ImageDataGenerator(rescale=1./255)
print(datagen)
train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')
validation_generator = datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [21]:
from video_module import Video,EmotionDetector


model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.save('emotion_detection_model.h5')
model = keras.models.load_model("emotion_detection_model.h5")
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=num_train // batch_size,
    epochs=num_epoch,
    validation_data=validation_generator,
    validation_steps=num_val // batch_size)


Epoch 1/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 61s 132ms/step - accuracy: 0.2612 - loss: 1.7946 - val_accuracy: 0.4175 - val_loss: 1.5015
Epoch 2/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3332 - loss: 1.5967 - val_accuracy: 0.4120 - val_loss: 1.5067
Epoch 3/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 64s 142ms/step - accuracy: 0.4202 - loss: 1.4899 - val_accuracy: 0.5034 - val_loss: 1.3008
Epoch 4/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.4586 - loss: 1.3331 - val_accuracy: 0.5014 - val_loss: 1.2955
Epoch 5/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 66s 148ms/step - accuracy: 0.4923 - loss: 1.3224 - val_accuracy: 0.5409 - val_loss: 1.2149
Epoch 6/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.5365 - loss: 1.2685 - val_accuracy: 0.5399 - val_loss: 1.2082
Epoch 7/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 70s 156ms/step - accuracy: 0.5324 - loss: 1.2229 - val_accuracy: 0.5599 - val_loss: 1.1582
Epoch 8/10
448/448 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5419 - loss: 1.2498 - 

In [19]:


class Video:
    def __init__(self, path_to_video):
        self.path = path_to_video
        self.cap = cv2.VideoCapture(path_to_video)
    
    def analyze(self, detector, display=False):
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break
            
            # Perform emotion detection on the frame
            result = detector.detect_emotions(frame)
            if display:
                # Display the frame with the detection results
                for emotion in result:
                    cv2.putText(frame, emotion, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.imshow('Video', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        
        self.cap.release()
        cv2.destroyAllWindows()


class EmotionDetector:
    def __init__(self, model):
        self.model = model
    
    def detect_emotions(self, frame):
        # Preprocess the frame as needed for your model
        # Here is a dummy implementation
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        frame = cv2.resize(frame, (48, 48))   
        frame = frame / 255.0 
        frame = frame.reshape(1, 48, 48, 1) 

        # Make predictions
        predictions = self.model.predict(frame)
        
        # Convert predictions to emotion labels
        # This is a dummy implementation, replace with your actual logic
        emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
        detected_emotions = [emotions[pred.argmax()] for pred in predictions]

        return detected_emotions 

In [39]:
path_to_video = "C:/Users/athen/OneDrive/Documents/Mini project 2/static/files/Flexpressions.mp4"
video = Video(path_to_video)
emotion_detector = EmotionDetector(model)
result = video.analyze(model, display=True)

NameError: name 'cv2' is not defined

In [ ]:
emotions_df = video.to_pandas(result)
emotions_df.head()

In [ ]:
# Predict whether a person show interest in a topic or not

positive_emotions = sum(emotions_df.happy) + sum(emotions_df.surprise)
negative_emotions = sum(emotions_df.angry) + sum(emotions_df.disgust) + sum(emotions_df.fear) + sum(emotions_df.sad)

if positive_emotions > negative_emotions:
    print("Person is interested")
elif positive_emotions < negative_emotions:
    print("Person is not interested")
else:
    print("Person is neutral")

In [7]:
from tensorflow.keras.models import load_model


model.save('emotion_detection_model.h5')
model = load_model('emotion_detection_model.h5')
def preprocess_frame(frame):
    if frame is None:
        return None
    try:  
        print("Original Frame Shape:", frame.shape)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        print("Grayscale Frame Shape:", frame.shape)
        frame = cv2.resize(frame, (48, 48))  
        print("Resized Frame Shape:", frame.shape)
        frame = frame / 255.0 
        frame = frame.reshape(1, 48, 48, 1) 
        print("Processed Frame Shape:", frame.shape)
        return frame
    except Exception as e:
        print("Error during frame preprocessing:", e)
        return None





In [6]:
video_path = 'C:/Users/athen/OneDrive/Documents/Mini project 2/static/files/Flexpressions.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

processed_frame = preprocess_frame(frame)
print("Processed Frame:", processed_frame)

    
emotion_prediction = model.predict(processed_frame)
out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

Processed Frame: None


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
camera=cv2.VideoCapture(0)
while True:
    success,frame=camera.read()
    if success:
        img=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        
        cv2.imshow("output",frame)
        k=cv2.waitKey(1)
        if k==ord("a"):
            camera.release()
            cv2.destroyAllWindows()
            break
            